In [5]:
!pip install pulp

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.2/16.4 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.5/16.4 MB 3.8 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/16.4 MB 5.8 MB/s eta 0:00:03
   --- ------------------------------------ 1.5/16.4 MB 7.3 MB/s eta 0:00:03
   ----- ---------------------------------- 2.2/16.4 MB 9.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.9/16.4 MB 10.3 MB/s eta 0:00:02
   --------- ------------------------------ 3.8/16.4 MB 11.5 MB/s eta 0:00:02
   ----------- ---------------------------- 4.6/16.4 MB 12.9 MB/s eta 0:00:01
   ------------- -------------------------- 5.7/16.4 MB 13.9 MB/s eta 0:00:01
   ---------------- ----------------------- 6.8/16.4 MB 15.1 MB/s eta 0:00:01
   ------------------- -------------------- 7.9/16.4 MB 16.4 MB/s eta 0:00:01
   ----

In [7]:
import pandas as pd
from pulp import *

In [35]:
# Import Nutrition
nutrition = pd.read_excel('Nutrition Facts.xlsx', index_col = 0, skiprows=1)
nutrition = nutrition.loc[:, ~nutrition.columns.str.contains('^Unnamed')]
nutrition = nutrition.dropna()

# Drop any rows where 'Ingredients' is NaN
nutrition = nutrition[nutrition["Ingredients"].notna()]

# Reset the index
nutrition.reset_index(drop=True, inplace=True)
nutrition.set_index("Ingredients", inplace=True)


nutrition

,Protein,Fat,Fibre,Salt,Sugar
Ingredients,,,,,
Chicken,0.100000,0.080000,0.001000,0.002,0.000
Beef,0.200000,0.100000,0.005000,0.005,0.000
Mutton,0.150000,0.110000,0.003000,0.007,0.000
Rice,0.000000,0.010000,0.100000,0.002,0.000
Wheat bran,0.040000,0.010000,0.150000,0.008,0.000
Corn,0.032927,0.012805,0.028049,0.000,0.045
Peanuts,0.258000,0.492000,0.085000,0.001,0.047


In [27]:
# Import Costs
costs = pd.read_excel('Costs.xlsx')
dict_costs = dict(zip(costs['Ingredients'], costs['Costs']))
costs

,Ingredients,Costs
0,Chicken,0.095
1,Beef,0.150
2,Mutton,0.100
3,Rice,0.002
4,Wheat bran,0.005
5,Corn,0.012
6,Peanuts,0.013


In [41]:
# Variables
variables = ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat bran', 'Corn', 'Peanuts']

# Initialize Class
model = LpProblem("Optimize your Protein Bar", LpMinimize)

# Create Decision Variables
x = LpVariable.dicts("Qty", [j for j in variables],
                     lowBound=0, upBound=None, cat='continuous')

# Define Objective Function
model += (lpSum([dict_costs[i] * x[i] for i in variables]))


# Add Constraints
model += (lpSum([x[i] for i in variables])) == 150
model += (lpSum([x[i] * nutrition.loc[i, 'Protein'] for i in variables])) >= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fat'] for i in variables])) <= 20
model += (lpSum([x[i] * nutrition.loc[i, 'Fibre'] for i in variables])) >= 6
model += (lpSum([x[i] * nutrition.loc[i, 'Salt'] for i in variables])) <= 3
model += (lpSum([x[i] * nutrition.loc[i, 'Sugar'] for i in variables])) <= 20
                                                   
                                                       
# Solve Model
model.solve()
print("Cost per Bar = {:,} $".format(round(value(model.objective), 2)))
print('\n' + "Status: {}".format(LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", round(v.varValue,2), 'g')

Cost per Bar = 10.24 $

Status: Optimal
Qty_Beef = 63.98 g
Qty_Chicken = 0.0 g
Qty_Corn = 0.0 g
Qty_Mutton = 0.0 g
Qty_Peanuts = 26.43 g
Qty_Rice = 0.0 g
Qty_Wheat_bran = 59.58 g


C:\Users\chint\anaconda3\Lib\site-packages\pulp\pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
